In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
import matplotlib.collections
from scipy.interpolate import interp1d
sns.set()
from results import load
from matplotlib.lines import Line2D

from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.axes_grid1.colorbar import colorbar
from functions import *
#constants
experiment_phase = {'p0':60000, 'p1':60000, 'p2':180000}
experiment_phase_labels = [*experiment_phase]

#avoid Font-problem in plots for ACM/IEEE papers
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

<ipython-input-1-ce831e714e12>:13: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid1.colorbar module was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use matplotlib.colorbar instead.
  from mpl_toolkits.axes_grid1.colorbar import colorbar


# Reading the results

In [2]:
import os


In [3]:
#config
experiment_name = "low"
all = None

providers = ["azure"] 
num_repetitions = 2
configs = os.listdir("results/low")


all = load(experiment_name, providers,configs,num_repetitions,True)

all.columns

Index(['RId', 'CIdOld', 'WL', 'CId', 'CStart', 'ECost', 'DLat', 'ELat', 'FOpt',
       'RFailed', 'Phase', 'RStart', 'EStart', 'EEnd', 'REnd', 'CNew', 'CPlat',
       'TLat', 'COs', 'PlatfromName', 'RInput', 'Provider', 'FEnv', 'Region',
       'RLat', 'BLat', 'RResult', 'run', 'sourceFile', 'RCode', 'RSuccessed',
       'HIdOld', 'HId'],
      dtype='object')

### Styles and Constants

In [4]:
#sns.set_palette(sns.color_palette("Paired"))

palette = sns.color_palette().as_hex()
style = {
    "aws":{
        "name":"AWS",
        "color":palette[8]
    },
    "ibm":{
        "name":"ICF",
        "color":palette[1]
    },
    "gcf":{
        "name":"GCF",
        "color":palette[2]
    },
    "azure":{
        "name":"MAF",
        "color":palette[4]
    },
    "CNew":{
        "New":"#0485d1",
        "Cold":"#0485d1",
        "Reused": "#fc5a50",
        "Warm": "#fc5a50"
    },
    #colors
    "Lats":{
        "Delivery Latency":"#d1ffbd",
        "Request-Response Latency":"#448ee4",
        "Execution Latency":"#25a36f",
        "Backward Response Delivery Latency":"#89a203",
        "Delivery":"#89a203",
        "Delivery Latency":"#89a203",
        "Request-Response":"#448ee4",
        "Execution":"#25a36f",
        "Backward Response Delivery":"#89a203",
        "DLat":"#89a203",
        "BLat":"#89a203",
        "RLat":"#448ee4",
        "ELat":"#25a36f"
    },
    "Phases":{
        "p0":"#74B52F",
        "p2":"#2F74B5",
        "p1":"#B52F74",
        "warmup":"#74B52F",
        "cooldown":"#2F74B5",
        "scaling":"#B52F74"
    },
    #Strings
    "Names":{
        "DLat":"Delivery Latency",
        "RLat":"Request-Response Latency",
        "ELat":"Execution Latency",
        "BLat":"Backward Response Delivery Latency",
        "p0":"warmup",
        "p1":"scaling",
        "p2":"cooldown",
        "HId":"VM",
        "CId":"Container",
        "HIdOld":"VM (old method)",
        "FEnv":"VM",
        "FOpt":"VM",
        "CIdOld":"Container",
    }
    
}

hmcmp = [(1,1,1,1)]+sns.color_palette("OrRd", 10)

sns.set(style="whitegrid")

legendShapes = []
legendNames = []
for x in ["RLat","ELat","DLat"]:
    c = style["Lats"][x]
    legendShapes.append(Line2D([0], [0], marker='s', lw=0,color=c,markerfacecolor=c, markersize=10))
    legendNames.append(x)

for x in ["New","Reused"]:
    c = style["CNew"][x]
    legendShapes.append(Line2D([0], [0], marker='s', lw=0,color=c,markerfacecolor=c, markersize=10))
    legendNames.append(x)

for x in ["p0","p1","p2"]:
    c = style["Phases"][x]
    legendShapes.append(Line2D([0], [0], marker='s', lw=0,color=c,markerfacecolor=c, markersize=10))
    legendNames.append(x)

for x in ["aws","ibm","gcf","azure"]:
    c = style[x]["color"]
    legendShapes.append(Line2D([0], [0], marker='s', lw=0,color=c,markerfacecolor=c, markersize=10))
    legendNames.append(x)



In [5]:
#this is important for phase based plots (leave it otherwise)
all = all[(all["Phase"] != "none")]
all["WL"].unique()

array(['5,5,0', '15,15,0', '0,0,1', '60,60,0'], dtype=object)

In [6]:
dlat_ylims = {
    "aws":{"0,0,0.5":[0,5],"0,0,1":[0,5],"0,0,2":[0,5],"60,60,0.5":[0,5],"60,60,1":[0,5],"60,60,0":[0,5]},
    "ibm":{"0,0,0.5":[0,5],"0,0,1":[0,5],"0,0,2":[0,5],"60,60,0.5":[0,5],"60,60,1":[0,5],"60,60,0":[0,5]},
    "azure":{"0,0,0.5":[0,30],"0,0,1":[0,30],"0,0,2":[0,30],"60,60,0.5":[0,30],"60,60,1":[0,30],"60,60,0":[0,30]},
    "gcf":{"0,0,0.5":[0,30],"0,0,1":[0,30],"0,0,2":[0,30],"60,60,0.5":[0,30],"60,60,1":[0,5],"60,60,0":[0,5]}
}
ylims = dlat_ylims

# Masterplot V2

In [7]:
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec
all_280 = all#[all["EStart"].isna() | all["EStart"] < 280000 ]
all_280 = all_280[all_280["run"]==1]
#Latency Limits
default = [0,60]
ylims = {
        "aws":{"0,0,0.5":default,"0,0,1":default,"0,0,2":default,"60,60,0.5":default,"60,60,1":default,"60,60,0":default},
        "ibm":{"0,0,0.5":default,"0,0,1":default,"0,0,2":default,"60,60,0.5":default,"60,60,1":default,"60,60,0":default},
        "azure":{"0,0,0.5":default,"0,0,1":default,"0,0,2":default,"60,60,0.5":default,"60,60,1":default,"60,60,0":default,"5,5,0":default,"15,15,0":default},
        "gcf":{"0,0,0.5":default,"0,0,1":default,"0,0,2":default,"60,60,0.5":default,"60,60,1":default,"60,60,0":default},
}


#Heatmap obergrenze
vmax=20
#Heatmap VM-cutoff (VMs mit weniger als x [%] auslastung  ) Reduziert verticale dichte und rechenzeit
cutoff=0.1

In [12]:
all_280.groupby(["Provider","WL"])["FOpt"].apply(lambda x:len(x.unique()))

Provider  WL     
azure     0,0,1      2
          15,15,0    1
          5,5,0      2
          60,60,0    2
Name: FOpt, dtype: int64

In [20]:
all_280.loc[all_280["FOpt"].isna(),"FOpt"] = "unk"

In [ ]:
heatmap_cache ={}

with PdfPages("reports/maf_all.pdf") as pdf:
    for workload in configs:
        for provider in providers:
            fig = plt.figure(figsize=(14,15))
            gs = fig.add_gridspec(ncols=2, nrows=2)
            plt.suptitle("Results for {} - {}".format(workload,style[provider]["name"]), fontsize=16,y=0.99)
            ax = fig.add_subplot(gs[0, 0:2])
            plotClientSideOverview(ax,all_280,provider,workload,style=style,ylim=ylims[provider][workload],
                                   xlim=[0,280],with_thruput=True,title="Client-side Impacts",legend=False)
            legendShapes = [
                Line2D([0], [0], color="gray", lw=2),
                Line2D([0], [0], marker='+', lw=0,color='indianred',markerfacecolor='indianred', markersize=10),
                Line2D([0], [0], color=style["Lats"]["RLat"], lw=2),
                Line2D([0], [0], color=style["Lats"]["ELat"], lw=2),
                Line2D([0], [0], marker='|', lw=0,color='blue',markerfacecolor='blue', markersize=10)
            ]
            legendNames = ["Target Workload","Failed Requests",style["Names"]["RLat"],style["Names"]["ELat"],"First VM Occurrence"]
            ax.legend(legendShapes, legendNames,
                loc='upper center', bbox_to_anchor=(0.5, 1.25),fancybox=True, shadow=True, ncol=2)
            ax.set_xlabel(None)
            ax = fig.add_subplot(gs[-1, 0])
            plotHeatmap(ax,all_280,provider,workload,style,selector="FOpt",cutoff=cutoff,xlim=[0,280],
                        legend=False,title="Provider-side Impacts",cmap=hmcmp,vmax=vmax,heatmap_cache=heatmap_cache)
            for _, spine in ax.spines.items():
                spine.set_visible(True)

            ax = fig.add_subplot(gs[-1, 1])
            plotHeatmap(ax,all_280,provider,workload,style,selector="CIdOld",cutoff=cutoff,xlim=[0,280],
                        legend=False,title="Provider-side Impacts",cmap=hmcmp,vmax=vmax,heatmap_cache=heatmap_cache)
            for _, spine in ax.spines.items():
                spine.set_visible(True)
            #plt.tight_layout()
            pdf.savefig(fig, bbox_inches="tight")
            fig.savefig("reports/overview_tall_{}_{}.pdf"
                .format(style[provider]["name"],workload)
                        , bbox_inches="tight"
            )
            plt.show()
            plt.close(fig)